In [1]:
import pandas as pd
from preprocess import process_cad_rads_labels

In [2]:
# data = pd.read_excel('CADRADS_exam.xlsx')
# data = data.iloc[:48]

# data = pd.read_excel('CADRADS_exam_v2.1.xlsx')
# data = pd.read_excel('CADRADS_exam_v2.2.xlsx')
# data = pd.read_excel('CADRADS_exam_v2.3.xlsx')
# data = pd.read_excel('CADRADS_exam_v2.4.xlsx')

# data = pd.read_excel('CADRADS_exam_addv3.0.xlsx')
# data = pd.read_excel('CADRADS_exam_addv3.1(외부병원).xlsx')
# data1 = pd.read_excel('240811_MYK_CADRADS_exam_addv3.1.xlsx', sheet_name=1)
# data2 = pd.read_excel('240811_MYK_CADRADS_exam_addv3.1.xlsx', sheet_name=2)
# data = pd.read_excel('240811_MYK_CADRADS_exam_addv3.1.xlsx', sheet_name=0)
# data = pd.read_excel('240818MYK_CADRADS_exam_addv3.2_20240814.xlsx', sheet_name=0)
# data = pd.read_excel('CCTA_addition240929.xlsx', sheet_name=0)
# data = pd.read_excel('CADRAD_internal추가n=50.xlsx', sheet_name=0)
# data = pd.read_excel('Internal_241201.xlsx', sheet_name=0)
data = pd.read_excel('External_241201_v1.0.xlsx', sheet_name=0)

In [13]:
data = pd.concat((data2, data1))

NameError: name 'data2' is not defined

In [3]:
data.head()

,No.,Unnamed: 1,CAD-RADS,CAC_available
0,1,"1. Coronary artery CT angiogram.\n LM, LAD,...",4B/P4,1
1,2,1. Coronary artery CT angiogram.\n p-LAD an...,4B/P4,1
2,3,1. Coronary artery CT angiogram.\n p-LAD: d...,3/P2,1
3,4,1. Coronary artery CT angiogram.\n p-RCA: d...,3/P3,1
4,5,1. Coronary artery CT angiogram.\n p-LAD: d...,1/1,1


In [12]:
data['Synthetic report'][0]

'Mild atherosclerosis with no significant stenosis in the coronary arteries.\n LM LCx unremarkable.\n mLAD1 40-50% stenosis with noncalcified plaque.\n m-dLAD2 patent stent without ISR.\n OM1 20-30% stenosis with calcified plaque.\n pRCA <20% stenosis with noncalcified plaque.\n\nSubendocardial low attenuation diffuse apical wall.\n - r/o subendocardial OMI in dLAD territory.\n\nNo definite evidence of PTE nor acute aortic syndrome.  '

In [14]:
def preprocess_text(text):
    if 'FINDING' in text:
        # FINDING이라는 키워드를 기준으로 텍스트를 분할합니다.
        parts = str(text).split('FINDING')
        
        # 'report'는 FINDING 앞의 모든 텍스트를 포함합니다.
        report = parts[0].strip()
        
        # 'label'은 FINDING 뒤의 텍스트에서 첫 번째 '['와 마지막 ']' 사이의 부분을 포함합니다.
        # 이때 'label' 정보가 반드시 존재한다고 가정합니다.
        label_part = parts[1].strip()
        label = label_part[label_part.find('[') + 1:label_part.rfind(']')].strip()
    else:
        report = text
        label = None
    return report, label

In [15]:
reports = []
labels = []
for i, d in enumerate(data['Synthetic report']):
    # print(i)
    report, label = preprocess_text(d)
    reports.append(report)
    labels.append(label)

KeyError: 'Synthetic report'

In [4]:
# data_processed = pd.DataFrame({"Report": reports, "Label": labels})

# data_processed = pd.DataFrame({"Report": reports, "Label": data['CADRAD확인수정_JKN']})
# data_processed = pd.DataFrame({"Report": data['Synthetic report'], "Label": data['CADRAD확인수정_JKN']})
# data_processed = pd.DataFrame({"Report": reports, "Label": data['MYK']})
# data_processed = pd.DataFrame({"Report": data['Synthetic Report Except Impression'], "Label": data['CADRADS MYK']})
# data_processed = pd.DataFrame({"Report": data['Synthetic report'], "Label": data['CADRAD']})
# data_processed = pd.DataFrame({"Report": data['Synthetic report'], "Label": data['CADRAD_BSH']})
# data_processed = pd.DataFrame({"Report": data['Synthetic report'], "Label": data['CADRAD확인수정_JKN']})
data_processed = pd.DataFrame({"Report": data['Unnamed: 1'], "Label": data['CAD-RADS']})
data_processed['Label'] = data_processed['Label'].astype(str)

In [5]:
processed_data = process_cad_rads_labels(data_processed, 'Label')
processed_data['Report'] = data_processed['Report']
processed_data.reset_index(inplace=True)

In [5]:
# 내부데이터 100개 중 수정
# processed_data.loc[11,'N']=1
# processed_data.loc[32,'Plaque Burden']=None

In [6]:
# 4는 4A로 간주
print(processed_data.loc[processed_data['CAD-RADS']=='4', 'CAD-RADS'])
processed_data.loc[processed_data['CAD-RADS']=='4', 'CAD-RADS']='4A'
print(processed_data.loc[processed_data['CAD-RADS']=='4', 'CAD-RADS'])

print(processed_data.loc[processed_data['CAD-RADS']=='None', 'CAD-RADS'])
processed_data.loc[processed_data['CAD-RADS']=='None', 'CAD-RADS']='0'

Series([], Name: CAD-RADS, dtype: object)
Series([], Name: CAD-RADS, dtype: object)
22    None
Name: CAD-RADS, dtype: object


In [7]:
# P0는 None으로 간주
print(processed_data.loc[processed_data['Plaque Burden']=='P0', 'Plaque Burden'])
processed_data.loc[processed_data['Plaque Burden']=='P0', 'Plaque Burden']='None'
print(processed_data.loc[processed_data['Plaque Burden']=='P0', 'Plaque Burden'])

Series([], Name: Plaque Burden, dtype: object)
Series([], Name: Plaque Burden, dtype: object)


In [8]:
processed_data['CAC_available'] = list(data['CAC_available'])

In [9]:
processed_data

,index,CAD-RADS,Plaque Burden,I,E,G,S,HRP,N,Report,CAC_available
0,0,4B,P4,0,0,0,0,0,0,"1. Coronary artery CT angiogram.\n LM, LAD,...",1
1,1,4B,P4,0,0,0,0,0,0,1. Coronary artery CT angiogram.\n p-LAD an...,1
2,2,3,P2,0,0,0,0,0,0,1. Coronary artery CT angiogram.\n p-LAD: d...,1
3,3,3,P3,0,0,0,0,0,0,1. Coronary artery CT angiogram.\n p-RCA: d...,1
4,4,1,None,0,0,0,0,0,0,1. Coronary artery CT angiogram.\n p-LAD: d...,1
5,5,1,None,0,0,0,0,0,0,1. Coronary artery CT angiogram.\n mLAD and...,1
6,6,5,None,0,0,0,0,0,0,S/P CABG - diffuse occlusion of graft vessel f...,0
7,7,2,P3,0,0,0,0,0,0,1. Coronary artery CT angiogram.\n m-RCA: d...,1
8,8,3,P3,0,0,0,0,0,0,1. Coronary artery CT angiogram.\n p-RCA: d...,1
9,9,5,None,0,0,0,0,0,0,1. Coronary artery CT angiogram.\n p- to d-...,0


In [10]:
# processed_data.to_csv('sample_processed_internal_test_50_v1.1.csv')
# processed_data.to_csv('sample_processed_v4.2_MYK_external.csv')
# processed_data.to_csv('sample_processed_internal_241201_v1.0.csv')
processed_data.to_csv('sample_processed_external_241201_v1.0.csv')

In [10]:
data = pd.read_csv('sample_processed_v3.0.csv')

In [28]:
for i, row in data.iterrows():
    data.loc[i,'Report'] = row['Report'].split('FINDING')[0].strip()

In [30]:
data.to_csv('sample_processed_v3.1.csv')